In [38]:
%%pyspark

df = spark.read.load('abfss://capture@splacceler5lmevhdeon4ym.dfs.core.windows.net/SeattlePublicLibrary/Checkouts_by_Title.csv', format='csv'
## If header exists uncomment line below
, header=True
)
display(df.limit(10))

In [39]:
%%pyspark

# Show Schema
df.printSchema()

In [40]:
%%pyspark

from pyspark.sql import SparkSession
from pyspark.sql.types import *

# Primary storage info
capture_account_name = 'splacceler5lmevhdeon4ym' # fill in your primary account name
capture_container_name = 'capture' # fill in your container name
capture_relative_path = 'SeattlePublicLibrary/Checkouts_by_Title.csv' # fill in your relative folder path

capture_adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (capture_container_name, capture_account_name, capture_relative_path)
print('Primary storage account path: ' + capture_adls_path)

In [41]:
%%pyspark

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, TimestampType
csvSchema = StructType([
  StructField('usage_class', StringType(), True),
  StructField('checkout_type', StringType(), True),
  StructField('material_type', StringType(), True), 
  StructField('checkout_year', IntegerType(), True),
  StructField('checkout_month', IntegerType(), True),
  StructField('checkouts', IntegerType(), True),   
  StructField('title', StringType(), True),
  StructField('creator', StringType(), True),
  StructField('subjects', StringType(), True),
  StructField('publisher', StringType(), True),
  StructField('publication_year', StringType(), True)   
])

CheckByTPI_capture_df = spark.read.format('csv').option('header', 'True').schema(csvSchema).load(capture_adls_path)

display(CheckByTPI_capture_df.limit(10))



In [42]:
%%pyspark

from pyspark.sql.functions import to_date, to_timestamp, col, date_format, current_timestamp
df_final = (CheckByTPI_capture_df.withColumn('loadDate', date_format(current_timestamp(), 'MM/dd/yyyy hh:mm:ss aa'))
                                 .withColumn("load_date", to_timestamp(col("loadDate"),"MM/dd/yyyy hh:mm:ss aa")).drop("loadDate")
)

In [43]:
%%pyspark

# Show Schema
df_final.printSchema()

display(df_final.limit(10))


In [44]:
%%pyspark

from pyspark.sql import SparkSession
from pyspark.sql.types import *

# Primary storage info
compose_account_name = 'splacceler5lmevhdeon4ym' # fill in your primary account name
compose_container_name = 'compose' # fill in your container name
compose_relative_path = 'SeattlePublicLibrary/CheckoutsByTitle/' # fill in your relative folder path

compose_adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (compose_container_name, compose_account_name, compose_relative_path)
print('Primary storage account path: ' + compose_adls_path)

In [45]:
%%pyspark

compose_parquet_path = compose_adls_path + 'checkouts.parquet'

print('parquet file path: ' + compose_parquet_path)

In [46]:
%%pyspark

df_final.write.parquet(compose_parquet_path, mode = 'overwrite')

In [48]:
%%sql

-- Create database SeattlePublicLibrary only if database with same name does not exist
CREATE DATABASE IF NOT EXISTS SeattlePublicLibrary

In [47]:
%%sql

-- Create table CheckoutsByTitlePhysicalItemsschemafinal only if table with same name does not exist
CREATE TABLE IF NOT EXISTS SeattlePublicLibrary.checkouts_by_title
(usage_class STRING
,checkout_type STRING
,material_type STRING
,checkout_year INT
,checkout_month INT
,checkouts INT
,title STRING
,creator STRING
,subjects STRING
,publisher STRING
,publication_year STRING
,load_date TIMESTAMP
)
USING PARQUET OPTIONS (path 'abfss://compose@splacceler5lmevhdeon4ym.dfs.core.windows.net/SeattlePublicLibrary/CheckoutsByTitle/checkouts.parquet')

In [23]:
%%sql

--DROP TABLE SeattlePublicLibrary.Checkouts_By_Title